# Goal

The purpose of this notebook is to provide an example use case of running the ioos/ioos_qc qartod library on a netCDF file.

In [1]:
# Install QC library
#!pip install git+git://github.com/ioos/ioos_qc.git #(only run if the ioos_qc library not installed)
from ioos_qc.config import NcQcConfig
from ioos_qc import qartod

/home/andrew/anaconda3/lib/python3.7/site-packages/xarray/core/merge.py:10: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  PANDAS_TYPES = (pd.Series, pd.DataFrame, pd.Panel)


In [2]:
# Other imports
import pandas as pd
import numpy as np
import xarray as xr
from datetime import datetime

from bokeh.layouts import gridplot
from bokeh.plotting import figure, show, output_file, output_notebook
output_notebook()

Loading BokehJS ...

## Check the netCDF dataset

In [4]:
pco2 = xr.open_dataset("pco2_netcdf_example.nc")

In [6]:
for dim in pco2.dims:
    print(dim)

spectrum
time


In [7]:
for var in pco2.variables:
    print(var)

obs
time
deployment
id
dcl_controller_timestamp
driver_timestamp
ingestion_timestamp
internal_timestamp
light_measurements
passed_checksum
port_timestamp
preferred_timestamp
provenance
record_time
record_type
thermistor_raw
unique_id
voltage_battery
absorbance_ratio_434
absorbance_ratio_620
pco2w_thermistor_temperature
absorbance_blank_434
absorbance_blank_620
pco2_seawater
absorbance_ratio_434_qc_executed
absorbance_ratio_434_qc_results
absorbance_ratio_620_qc_executed
absorbance_ratio_620_qc_results
pco2w_thermistor_temperature_qc_executed
pco2w_thermistor_temperature_qc_results
pco2_seawater_qc_executed
pco2_seawater_qc_results
lat
lon


## Setup & Run a single QC test

In [8]:
# Create the config object
config = {
    'pco2_seawater': {
        'qartod': {
            'gross_range_test': {
                'suspect_span': [200, 600],
                'fail_span': [0, 1200]
            }
        }
    }
}

qc = NcQcConfig(config)

In [9]:
# To run the qc on a netCDF file, call the path to the file, not the load netCDF dataset
qc_gross_range = qc.run("pco2_netcdf_example.nc")

In [10]:
# Check that the output is an OrderedDict and ran correctly
print(qc_gross_range)

OrderedDict([('pco2_seawater', OrderedDict([('qartod', OrderedDict([('gross_range_test', masked_array(data=[4, 4, 3, ..., 1, 1, 1],
             mask=False,
       fill_value=999999,
            dtype=uint8))]))]))])


In [38]:
# Method to plot QC results using Bokeh
def plot_ncresults(ncdata, var_name, results, title, test_name):

    time = np.array(ncdata.variables['time'])
    obs = np.array(ncdata.variables[var_name])
    qc_test = results[var_name]['qartod'][test_name]

    qc_pass = np.ma.masked_where(qc_test != 1, obs)
    qc_suspect = np.ma.masked_where(qc_test != 3, obs)
    qc_fail = np.ma.masked_where(qc_test != 4, obs)
    qc_notrun = np.ma.masked_where(qc_test != 2, obs)

    p1 = figure(x_axis_type="datetime", title=test_name + ' : ' + title)
    p1.grid.grid_line_alpha=0.3
    p1.xaxis.axis_label = 'Time'
    p1.yaxis.axis_label = 'Observation Value'

    p1.line(time, obs,  legend='obs', color='#A6CEE3')
    p1.circle(time, qc_notrun, size=2, legend='qc not run', color='gray', alpha=0.2)
    p1.circle(time, qc_pass, size=4, legend='qc pass', color='green', alpha=0.5)
    p1.circle(time, qc_suspect, size=4, legend='qc suspect', color='orange', alpha=0.7)
    p1.circle(time, qc_fail, size=6, legend='qc fail', color='red', alpha=1.0)

    #output_file("qc.html", title="qc example")

    show(gridplot([[p1]], plot_width=800, plot_height=400))

In [39]:
# Just a quick note: OOI time data is in seconds since 1900-01-01, so that requires some further 
plot_ncresults(pco2, 'pco2_seawater', qc_gross_range, 'pCO2 seawater', 'gross_range_test')